In [5]:
import numpy as np

# Parameters (These can be adjusted as needed)
w = 1  # weight for gap cost
q = 2  # penalty parameter
e = 1  # gap extension penalty
p = 3  # non-canonical splicing penalty

# Gap cost functions
def gamma_c(l):
    if l > 0:
        return 0.01 * w * l + 0.5 * np.log2(l)
    else:
        return min(0.01 * w * abs(l), np.log2(abs(l)))

def gamma_a(l):
    if l > 0:
        return q + l * e
    else:
        return min(q + abs(l) * e, p)

# Reference sequence (example, replace with actual sequence)
T = "AGGTACGTCGACGTAG"

# Functions to compute d(i) and a(i)
def d(i):
    if i + 3 < len(T) and T[i+1:i+4] in ["GTA", "GTG"]:
        return 0
    elif i + 3 < len(T) and T[i+1:i+4] in ["GTC", "GTT"]:
        return p / 2
    else:
        return p

def a(i):
    if i - 2 >= 0 and T[i-2:i+1] in ["CAG", "TAG"]:
        return 0
    elif i - 2 >= 0 and T[i-2:i+1] in ["AAG", "GAG"]:
        return p / 2
    else:
        return p

# Scoring function s(i, j)
def s(i, j):
    # Placeholder scoring function, should be replaced with actual scoring
    return 1 if T[i] == T[j] else -1

# Spliced alignment with DP
def spliced_alignment(query, reference):
    m = len(query)
    n = len(reference)
    
    # Initialize matrices
    H = np.zeros((m+1, n+1))
    E = np.zeros((m+1, n+1))
    F = np.zeros((m+1, n+1))
    E_ = np.zeros((m+1, n+1))
    
    # Fill DP tables
    for i in range(1, m+1):
        for j in range(1, n+1):
            H[i, j] = max(H[i-1, j-1] + s(i-1, j-1), E[i, j], F[i, j], E_[i, j] - a(i))
            if i + 1 <= m:
                E[i+1, j] = max(H[i, j] - q, E[i, j] - e)
            if j + 1 <= n:
                F[i, j+1] = max(H[i, j] - q, F[i, j] - e)
            if i + 1 <= m:
                E_[i+1, j] = max(H[i, j] - d(i) - gamma_a(i), E_[i, j] - e)
    
    return H

# Example query and reference sequences
query = "AGTCGACGT"
reference = "AGGTACGTCGACGTAG"

# Run the spliced alignment
H = spliced_alignment(query, reference)

# # Display the alignment score matrix
# import ace_tools as tools; tools.display_dataframe_to_user(name="Spliced Alignment Score Matrix", dataframe=H)

# # Output the score matrix
# H


In [6]:
print(H)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  2.  1. -1. -1.  0.  1. -1. -1.  1. -1.  0.  1. -1. -1.  2.]
 [ 0.  0.  1.  3.  1.  0. -1.  1.  0. -2.  0.  0. -2.  1.  0. -2.  0.]
 [ 0.  0. -1.  1.  4.  2.  1.  0.  2.  0. -1. -1. -1. -1.  2.  0. -1.]
 [ 0.  1. -1.  0.  2.  5.  3.  2.  1.  1. -1.  0. -2. -2.  0.  3.  1.]
 [ 0.  0.  0. -1.  1.  3.  6.  4.  3.  2.  1.  0.  1. -1. -1.  1.  2.]
 [ 0.  0.  1.  1.  0.  2.  4.  7.  5.  4.  3.  2.  1.  2.  0.  0.  2.]
 [ 0.  0. -1.  0.  2.  1.  3.  5.  8.  6.  5.  4.  3.  2.  3.  1.  0.]
 [ 0.  0. -1. -2.  0.  1.  2.  4.  6.  9.  7.  6.  5.  4.  3.  2.  1.]]
